# Introdução

<p>
Apresentar o contexto do consumo de energia elétrica no Brasil,
o objetivo do estudo e as etapas que serão realizadas ao longo do notebook.
</p>

In [523]:
import pandas as pd
import calendar
import gzip

# Data Collection and Loading

The datasets used in this project were publicly made available at http://leandrolessa.com.br/datasets/.

They bring together information related to electricity consumption in Brazil and the geographic characteristics of the states, enabling exploratory data analysis, regional studies, and time series modeling.

##### consumo_energia_eletrica.csv
This historical dataset comprises electricity consumption data collected over multiple years and aggregated by state, making it suitable for exploratory data analysis and time series studies.

##### estado_regiao.csv
This dataset provides structured information on Brazilian states, covering state identification, their respective geographic regions, and the country.

##### populacao.csv
This dataset contains population estimates for Brazil by state, covering the period from 1991 to 2021.

In [524]:
# Loading data from CSV files into pandas DataFrames
# TODO tratar erro utf8 ao carregar o df dos estados

consumption_df = pd.read_csv('./consumo_energia_eletrica.csv')
states_df = pd.read_csv('./estado_regiao.csv', sep=';')


with gzip.open('./populacao.csv', 'rb') as f_in:
    population_df = pd.read_csv(f_in)

# del population_df['populacao_economicamente_ativa']


# Initial Exploratory Analysis

An initial exploratory analysis of the electricity consumption DataFrame indicates that it contains six variables — ano, mês, sigla_uf, tipo_consumo, numero_consumidores and consumo — totaling 39,897 records.

1,017 duplicate records were identified, corresponding to approximately 2.5% of the dataset. The field numero_consumidores contains 12,960 missing values, representing about 32.5% of the records, which requires careful consideration in subsequent analyses.

Regarding the states DataFrame, the fields sigla, estado, and regiao are relevant for the analysis, whereas the id_estado and pais fields will not be used. The DataFrame initially presented a UTF-8 encoding issue, which was promptly resolved.

### Energy consumption

In [525]:
consumption_df.shape

(39897, 6)

In [526]:
consumption_df.dtypes

ano                      int64
mes                      int64
sigla_uf                object
tipo_consumo            object
numero_consumidores    float64
consumo                  int64
dtype: object

In [527]:
consumption_df.head()

,ano,mes,sigla_uf,tipo_consumo,numero_consumidores,consumo
0,2004,1,TO,Total,NaN,65876
1,2004,1,BA,Total,NaN,1444451
2,2004,1,PR,Total,NaN,1596274
3,2004,1,RS,Total,NaN,1780912
4,2004,1,GO,Total,NaN,630624


#### check for null fields
Based on previous analysis, it became clear that the `numero_consumidores` field contains null values. In this case, I will analyze and try to discover if there are other fields with zero values.

In [528]:
null_df = consumption_df.isnull()
print(null_df.sum())
print(f'\nTotal number of missing values for the numero_consumidores field: {null_df['numero_consumidores'].sum()}')

ano                        0
mes                        0
sigla_uf                   0
tipo_consumo               0
numero_consumidores    12960
consumo                    0
dtype: int64

Total number of missing values for the numero_consumidores field: 12960


In [529]:
# filter numero_consumidores null
consumption_null_filtered = consumption_df[consumption_df['numero_consumidores'].isnull()]
consumption_type_nulls = consumption_null_filtered['tipo_consumo'].unique()

print("Consumption types with null number of consumers:", consumption_type_nulls)

Consumption types with null number of consumers: ['Total' 'Cativo']


##### <strong>After analyzing, I realized that the consumer numbers (`numero_consumidores`) that contain null consumption type data (`tipo_consumo`) are Cativo and Total.</strong>

#### check for duplicate records

In [530]:
duplicated_consumption_df = consumption_df.duplicated()
print(f'\nTotal number of duplicate lines in the energy consumption dataframe: {duplicated_consumption_df.sum() }')


Total number of duplicate lines in the energy consumption dataframe: 1017


#### show the types of consumption

In [531]:
print(consumption_df['tipo_consumo'].unique())

['Total' 'Cativo' 'Outros' 'Comercial' 'Industrial' 'Residencial']


### States

In [532]:
states_df.shape

(27, 5)

In [533]:
states_df.dtypes

id_estado     int64
sigla        object
estado       object
regiao       object
pais         object
dtype: object

In [534]:
states_df.head(27)

,id_estado,sigla,estado,regiao,pais
0,1,AC,Acre,Norte,Brasil
1,2,AL,Alagoas,Nordeste,Brasil
2,3,AP,Amapá,Norte,Brasil
3,4,AM,Amazonas,Norte,Brasil
4,5,BA,Bahia,Nordeste,Brasil
5,6,CE,Ceará,Nordeste,Brasil
6,7,DF,Distrito Federal,Centro-Oeste,Brasil
7,8,ES,Espírito Santo,Sudeste,Brasil
8,9,GO,Goiás,Centro-Oeste,Brasil
9,10,MA,Maranhão,Nordeste,Brasil


In [535]:
# search for null fields

null_df = states_df.isnull()
print(null_df.sum())

id_estado    0
sigla        0
estado       0
regiao       0
pais         0
dtype: int64


### Population

In [536]:
population_df.shape

(837, 4)

In [537]:
population_df.dtypes

sigla_uf                           object
ano                                 int64
populacao                           int64
populacao_economicamente_ativa    float64
dtype: object

In [538]:
population_df.head()

,sigla_uf,ano,populacao,populacao_economicamente_ativa
0,AC,1991,417102,NaN
1,AL,1991,2512658,NaN
2,AM,1991,2102766,NaN
3,AP,1991,289035,NaN
4,BA,1991,11867336,NaN


In [539]:
null_df = population_df.isnull()

print(null_df.sum())

sigla_uf                            0
ano                                 0
populacao                           0
populacao_economicamente_ativa    567
dtype: int64


In [540]:
population_df['sigla_uf'].unique()

array(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
       'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
       'RS', 'SC', 'SE', 'SP', 'TO'], dtype=object)

# Handling Duplicate Data

As observed in the initial exploratory analysis, duplicate records were identified in the electricity consumption DataFrame. After removing the duplicates, the DataFrame contains 38.880 records.

In [541]:
duplicated_df = consumption_df[duplicated_consumption_df]
print(duplicated_df)

        ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
38880  2023   11       MS       Outros              99710.0   122350
38881  2023   11       RN       Outros              77359.0   112767
38882  2023   11       PE       Outros             157848.0   256509
38883  2023   11       ES       Outros             234985.0   180315
38884  2023   11       AC       Outros              23068.0    26323
...     ...  ...      ...          ...                  ...      ...
39892  2023   12       BA  Residencial            5932740.0   718461
39893  2023   12       PA  Residencial            2614106.0   478726
39894  2023   12       AC  Residencial             245699.0    58301
39895  2023   12       RN  Residencial            1390080.0   210119
39896  2023   12       SC  Residencial            2719791.0   655389

[1017 rows x 6 columns]


#### remove duplicate rows

In [542]:
consumption_transformed_df = consumption_df.drop_duplicates()

validate_duplicated_df = consumption_transformed_df.duplicated()
print(f'\nTotal number of duplicate lines in the energy consumption dataframe after remove duplicates: {validate_duplicated_df.sum() }')

consumption_transformed_df.shape


Total number of duplicate lines in the energy consumption dataframe after remove duplicates: 0


(38880, 6)

# Handling Missing Data

As previously analyzed, `numero_consumidores` contains missing/null data, and it was identified that this missing data exists only for the `Cativo` and `Total` consumption types. I will analyze the data and understand the reason for the zero values ​​and try to find some relationship; if I am unsuccessful, I will remove the missing data.

#### tipo_consumo Cativo

In [543]:
# function to count tipo_consumo Cativo missing values
def count_consumpion_type_captive(df, consumption_type): 
    df = df[df["tipo_consumo"] == consumption_type]    
    return df["numero_consumidores"].isna().sum()

count_captive = count_consumpion_type_captive(consumption_transformed_df, 'Cativo')
print(f'\nTotal number of missing values for the numero_consumidores field with Cativo tipo_consumo: {count_captive}')

# removing tipo_consumo Cativo from dataframe
consumption_transformed_df = consumption_transformed_df[~((consumption_transformed_df["tipo_consumo"] == "Cativo") & (consumption_transformed_df["numero_consumidores"].isna()))]

count_captive = count_consumpion_type_captive(consumption_transformed_df, 'Cativo')
print(f'\nTotal number of missing values for the numero_consumidores field with Cativo tipo_consumo: {count_captive}')


Total number of missing values for the numero_consumidores field with Cativo tipo_consumo: 6480

Total number of missing values for the numero_consumidores field with Cativo tipo_consumo: 0


<strong>When analyzing the dataframe records with zero values ​​for captive consumption, I realized that, although it was possible to estimate the number of consumers using various statistical techniques, this approach is not adequate. This is because captive consumption encompasses distinct categories—residential, commercial, and small industries—which are separated in the dataframe. Therefore, inferring values ​​could generate distortions and incorrect conclusions. Thus, I will choose to remove the captive consumption records from the analysis.</strong>

#### tipo_consumo Total

In [544]:
# count
count_captive = count_consumpion_type_captive(consumption_transformed_df, 'Total')
print(f'\nTotal number of missing values for the numero_consumidores field with Total tipo_consumo: {count_captive}')


Total number of missing values for the numero_consumidores field with Total tipo_consumo: 6480


In [545]:
# filter for the state of Pernambuco (PE) in February 2022, and then divided into two groups: one showing the total consumption and the other showing all the other types of consumption. 
    
consumption_pe_22 = consumption_transformed_df.query(
    "ano == 2022 and mes == 2 and sigla_uf == 'PE'"
)

total_df = consumption_pe_22[consumption_pe_22['tipo_consumo'] == 'Total']
others_df = consumption_pe_22[consumption_pe_22['tipo_consumo'] != 'Total']

print("Total consumption:")
print(total_df)

print("\nOther consumption types:")
print(others_df)

# sum comsumptions
total_consumo_others = others_df['consumo'].sum()
total_consumo = total_df['consumo'].sum()

# print total
print("\nTotal consumption for other types:", total_consumo_others)
print("\nTotal consumption for total types:", total_consumo)

Total consumption:
        ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
35031  2022    2       PE        Total                  NaN  1282724

Other consumption types:
        ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
35679  2022    2       PE       Outros             171780.0   255068
35995  2022    2       PE    Comercial             226681.0   248323
36325  2022    2       PE   Industrial               4821.0   321438
36652  2022    2       PE  Residencial            3482337.0   457895

Total consumption for other types: 1282724

Total consumption for total types: 1282724


In [546]:
# filter for the state of Pernambuco (PE) in February 2022, and then divided into two groups: one showing the total consumption and the other showing all the other types of consumption. 
    
consumption_pe_22 = consumption_transformed_df.query(
    "ano == 2021 and mes == 3 and sigla_uf == 'CE'"
)

total_df = consumption_pe_22[consumption_pe_22['tipo_consumo'] == 'Total']
others_df = consumption_pe_22[consumption_pe_22['tipo_consumo'] != 'Total']

print("Total consumption:")
print(total_df)

print("\nOther consumption types:")
print(others_df)

# sum comsumptions
total_consumo_others = others_df['consumo'].sum()
total_consumo = total_df['consumo'].sum()

# print total
print("\nTotal consumption for other types:", total_consumo_others)
print("\nTotal consumption for total types:", total_consumo)

Total consumption:
        ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
33102  2021    3       CE        Total                  NaN  1015890

Other consumption types:
        ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
33775  2021    3       CE       Outros             738531.0   215326
34097  2021    3       CE    Comercial             203806.0   184112
34416  2021    3       CE   Industrial               7808.0   207590
34739  2021    3       CE  Residencial            3436862.0   408862

Total consumption for other types: 1015890

Total consumption for total types: 1015890


In [547]:
# separate the Total consumption types from the other types

consumption_transformed_df = consumption_transformed_df[consumption_transformed_df["tipo_consumo"] != "Total"]

consumption_total_df = consumption_transformed_df.groupby(
    ["ano", "mes", "sigla_uf"], as_index=False
).agg({
    "numero_consumidores": "sum",
    "consumo": "sum"
})

consumption_total_df["tipo_consumo"] = "Total"

consumption_total_df = consumption_total_df[["ano", "mes", "sigla_uf", "tipo_consumo", "numero_consumidores", "consumo"]]

# show tipo_consumo from consumption_total_df
print(consumption_transformed_df['tipo_consumo'].unique())

# show tipo_consumo from consumption_transformed_df
print(consumption_total_df.head())
print(consumption_total_df['tipo_consumo'].unique())

['Outros' 'Comercial' 'Industrial' 'Residencial']
    ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
0  2004    1       AC        Total             126275.0    34839
1  2004    1       AL        Total             637016.0   285489
2  2004    1       AM        Total             543155.0   274773
3  2004    1       AP        Total             114101.0    43084
4  2004    1       BA        Total            3502266.0  1444449
['Total']


In [548]:
# filter for the state of Pernambuco (PE) in February 2022, and then divided into two groups: one showing the total consumption and the other showing all the other types of consumption. 
    
consumption_pe_22 = consumption_transformed_df.query(
    "ano == 2022 and mes == 2 and sigla_uf == 'PE'"
)

consumption_total_pe_22 = consumption_total_df.query(
    "ano == 2022 and mes == 2 and sigla_uf == 'PE'"
)

others_df = consumption_pe_22[consumption_pe_22['tipo_consumo'] != 'Total']
total_df = consumption_total_pe_22[consumption_total_pe_22['tipo_consumo'] == 'Total']

print("Total consumption:")
print(total_df)

print("\nOther consumption types:")
print(others_df)

# sum comsumptions
total_consumo_others = others_df['consumo'].sum()
total_consumo = total_df['consumo'].sum()

# print total
print("\nTotal consumption for other types:", total_consumo_others)
print("\nTotal consumption for total types:", total_consumo)

print("\nTotal numero_consumidores for other types:", others_df['numero_consumidores'].sum())
print("\nTotal numero_consumidores for total types:", total_df['numero_consumidores'].sum())

Total consumption:
       ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
5874  2022    2       PE        Total            3885619.0  1282724

Other consumption types:
        ano  mes sigla_uf tipo_consumo  numero_consumidores  consumo
35679  2022    2       PE       Outros             171780.0   255068
35995  2022    2       PE    Comercial             226681.0   248323
36325  2022    2       PE   Industrial               4821.0   321438
36652  2022    2       PE  Residencial            3482337.0   457895

Total consumption for other types: 1282724

Total consumption for total types: 1282724

Total numero_consumidores for other types: 3885619.0

Total numero_consumidores for total types: 3885619.0


<strong>It was concluded that the numero_consumidores for the Total consumption type is empty because it represents the sum of all other consumption types. The dataframe will now be separated into two: one containing only the Total consumption type and the other containing all the other consumption types.</strong>

In [549]:
# remove populacao_economicamente_ativa column

del population_df['populacao_economicamente_ativa']

<strong>Regarding the economically active population field (`populacao_economicamente_ativa`) , I didn't find the initial data reliable, so I'm going to discard it from the dataframe and not pursue that approach.</strong>.

# Standardization and Adjustments

The DataFrame was merged with the states DataFrame, incorporating the `state` and `region` columns. The `month` column was updated to show the full month name instead of the numeric value. The `numero_consumidores` column was converted to an integer type, and the column names were updated for improved clarity.

In [550]:
# TODO perform this treatments before separating the dataframes

def merge_with_states(df):
    return pd.merge(
        df,
        states_df[['estado','regiao','sigla']],
        left_on='sigla_uf',
        right_on='sigla'
    )

def transform_standardization(df):
    # transform month numer by name
    df['mes'] = df['mes'].apply(lambda x: calendar.month_name[x])
    
    # cast numero_consumidores to int64
    df['numero_consumidores'] = df['numero_consumidores'].astype('Int64')
    
    # delete sigla column
    del df['sigla']
    
    # columns rename
    return df.rename(columns={
        'ano': 'year',
        'mes': 'month',
        'sigla_uf': 'uf',
        'tipo_consumo':  'type_consumption',
        'numero_consumidores': 'qty_consumers',
        'consumo': 'consumption',
        'estado': 'state',
        'regiao': 'region',
    })   

In [551]:
# merge with estados
consumption_transformed_df = merge_with_states(consumption_transformed_df)
print(f'consumption_transformed_df: {consumption_transformed_df.columns}')

consumption_total_df = merge_with_states(consumption_total_df)
print(f'consumption_total_df: {consumption_total_df.columns}')

consumption_transformed_df: Index(['ano', 'mes', 'sigla_uf', 'tipo_consumo', 'numero_consumidores',
       'consumo', 'estado', 'regiao', 'sigla'],
      dtype='object')
consumption_total_df: Index(['ano', 'mes', 'sigla_uf', 'tipo_consumo', 'numero_consumidores',
       'consumo', 'estado', 'regiao', 'sigla'],
      dtype='object')


In [552]:
consumption_transformed_df = transform_standardization(consumption_transformed_df)
consumption_total_df = transform_standardization(consumption_total_df)

print(consumption_transformed_df.columns)
print(consumption_total_df.columns)

Index(['year', 'month', 'uf', 'type_consumption', 'qty_consumers',
       'consumption', 'state', 'region'],
      dtype='object')
Index(['year', 'month', 'uf', 'type_consumption', 'qty_consumers',
       'consumption', 'state', 'region'],
      dtype='object')


# Creating Display Functions

<p>Criar funções reutilizáveis para geração de gráficos, permitindo análises flexíveis por região, ano ou
tipo de consumidor.</p>

# Performing the 15 Analyses

Realizar no mínimo 15 análises explorando aspectos temporais, regionais, por estado, por tipo de
consumidor e correlações. Cada gráfico deve conter descrição e interpretação.

# General Interpretation

Consolidar tendências, padrões e diferenças regionais observadas nos dados.

# Summary of Key Insights

Apresentar os principais achados de forma clara e objetiva em formato Markdown.

# Conclusion

Refletir sobre o processo analítico, desafios enfrentados e aprendizados adquiridos ao longo do
projeto.